In [ ]:
import torch
import numpy as np
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor


def cell2patch(wsi_name):
    cell_names = os.listdir(os.path.join(root_path, wsi_name))
    # cell_name = cell_names[0]
    # print(cell_name.split('-')[-1])
    feat_per_patch = {}
    for cell_name in cell_names:
        tmp = cell_name.split('_')
        patch_name = tmp[-1]
        cell_cat = tmp[0]
        cell_embed = np.load(os.path.join(root_path, wsi_name, cell_name))
        if patch_name not in feat_per_patch.keys():
            feat_per_patch[patch_name] = [cell_embed]
        else:
            feat_per_patch[patch_name].append(cell_embed)
    patch_embeds = []
    for patch_name in feat_per_patch.keys():
        patch_embed = np.stack(feat_per_patch[patch_name]) # [M, 256]
        patch_embeds.append(patch_embed) # [N, M~, 256]
    torch.save(patch_embeds, os.path.join(output_path, wsi_name+'.pt'))
    
root_path = '/mnt/tmp/TCT_DATA/rtdetr_feats_20240822_embed_x7/img_feats/embed6'
output_path = '/data/wsi/TCTGC10k-Cell/rtdetr-v1-2025.1.1'
wsi_names = os.listdir(root_path)
# wsi_name = '1914834909-01-HSIL'
wsi_names = os.listdir(root_path)
with ThreadPoolExecutor(max_workers=5) as executor:
    # map方法会异步执行square函数，对numbers列表中的每个元素进行处理
    futures = [executor.submit(cell2patch, wsi_name) for wsi_name in wsi_names]

    # 计数器
    completed_tasks = 0
    # 循环检查任务状态
    while completed_tasks < len(futures):
        for future in futures:
            if future.done():
                completed_tasks += 1
                print(f"Task completed: {future.result()}")  # 打印已完成任务的结果
        print(f"Progress: {completed_tasks}/{len(futures)}")
        time.sleep(0.5)  # 稍作延迟，避免频繁打印



Progress: 0/9717


In [10]:
import pandas as pd
import torch
import os

labels1_path = '/data/wsi/TCTGC50k-labels/TCTGC50k-volume1-labels-path.csv'
df1 = pd.read_csv(labels1_path)
wsi_names = [str(wsi_name).replace('.pt', '') for wsi_name in df1['wsi_id']]
print(len(wsi_names))

cell_feat_path = '/mnt/tmp/TCT_DATA/rtdetr_feats_20240822_embed_x7/img_feats/embed6'
cell_wsi_names = os.listdir(cell_feat_path)
print(len(cell_wsi_names))
diff1 = set(wsi_names) - set(cell_wsi_names)
# print(len(diff1))
diff1 = list(diff1)
torch.save(diff1, 'diff1.pt')

9994
9717


In [14]:
print(len(df1))
print(df1['wsi_label'].value_counts())

9994
wsi_label
NILM      5889
ASC-US    1226
LSIL       901
BV         860
M          371
ASC-H      293
HSIL       241
T          116
AGC         97
Name: count, dtype: int64
